In [1]:
import osmnx as ox 
ox.config(use_cache=True, log_console=True)
ox.__version__

'1.1.1'

In [2]:
place = {'city':'San Francisco', 'state':'California', 'country':'USA'}
G = ox.graph_from_place(place,
                        network_type='drive',
                        simplify=True,
                        retain_all=False, 
                        truncate_by_edge=True,
                        clean_periphery=True)

In [3]:
G = ox.add_edge_speeds(G)
G = ox.add_edge_travel_times(G)

In [4]:
nodes, edges = ox.graph_to_gdfs(G)
edges['highway'].astype(str).value_counts()

residential                        17765
tertiary                            3707
secondary                           2610
primary                             1534
unclassified                         558
motorway_link                        150
motorway                             126
primary_link                         111
secondary_link                        83
trunk                                 47
tertiary_link                         30
living_street                         28
trunk_link                            27
['residential', 'unclassified']       23
busway                                19
['residential', 'tertiary']            9
['tertiary', 'unclassified']           7
['secondary', 'tertiary']              4
['secondary', 'living_street']         2
['secondary', 'primary']               1
['primary_link', 'secondary']          1
Name: highway, dtype: int64

## filter out motorway edges

In [5]:
e = [(u, v, k) for u, v, k, d in G.edges(keys=True, data=True) if 'motorway' in d['highway']]

In [6]:
# remove motorway edges then retain only the largest strongly connected component
G.remove_edges_from(e)
G = ox.utils_graph.get_largest_component(G, strongly=True)

In [7]:
nodes, edges = ox.graph_to_gdfs(G)
edges['highway'].astype(str).value_counts()

residential                        17552
tertiary                            3614
secondary                           2610
primary                             1530
unclassified                         528
primary_link                         102
secondary_link                        82
trunk                                 37
tertiary_link                         30
living_street                         21
['residential', 'unclassified']       19
busway                                19
trunk_link                            19
['residential', 'tertiary']            9
['tertiary', 'unclassified']           7
['secondary', 'tertiary']              4
['secondary', 'living_street']         2
['secondary', 'primary']               1
['primary_link', 'secondary']          1
Name: highway, dtype: int64

## save graphs to disk

save directed graph to disk

In [8]:
ox.save_graphml(G, filepath='data/network/sf-directed-no-fwy.graphml')

In [9]:
nodes = ox.graph_to_gdfs(G, edges=False, node_geometry=False)
nodes.to_csv('data/network/sf-directed-no-fwy-nodelist.csv', index=True, encoding='utf-8')

save undirected representation of graph to disk

In [10]:
G_undir = ox.get_undirected(G)

In [11]:
ox.save_graphml(G_undir, filepath='data/network/sf-undirected-no-fwy.graphml')

In [12]:
nodes = ox.graph_to_gdfs(G_undir, edges=False, node_geometry=False)
nodes.to_csv('data/network/sf-undirected-no-fwy-nodelist.csv', index=True, encoding='utf-8')